In [2]:
# For create venv and install requirements
! make create-venv
! make pip-update
! make activate-venv
! make install-requirements

python3 -m venv venv
venv/bin/pip install --upgrade pip
source ./venv/bin/activate
make: source: No such file or directory
make: *** [activate-venv] Error 1
venv/bin/pip install -r requirements.txt


In [1]:
from dependency_injector import containers, providers
from dependency_injector.wiring import Provide, inject
from unittest import mock

In [62]:
class ApiClient:

    def __init__(self, api_key: str, timeout: int) -> None:
        print('INIT APICLIENT')
        self.api_key = api_key  # <-- dependency is injected
        self.timeout = timeout  # <-- dependency is injected

In [63]:
class Service:

    def __init__(self, api_client: ApiClient) -> None:
        print('INIT SERVICE')
        self.api_client = api_client  # <-- dependency is injected

In [64]:
class Container(containers.DeclarativeContainer):

    config = providers.Configuration()

    api_client = providers.Singleton(
        ApiClient,
        api_key=config.api_key,
        timeout=config.timeout,
    )

    service = providers.Factory(
        Service,
        api_client=api_client,
    )


@inject
def main(service: Service = Provide[Container.service]) -> None:
    print('INIT MAIN')

In [70]:
container = Container()
container.wire(modules=[__name__])

main()  # <-- dependency is injected automatically

with container.api_client.override(mock.Mock()):
    main()


INIT APICLIENT
INIT SERVICE
INIT MAIN
INIT SERVICE
INIT MAIN


In [34]:
class Uber:
    def __init__(self, car_type, amount_door) -> None:
        self.car_type: str = car_type
        self.amount_door: int = amount_door
        
    def __str__(self) -> str:
        return f'car type: {self.car_type} {self.amount_door} amount door. '

class Passenger:
    def __init__(self, origin, destiny) -> None:
        self.origin: str = origin
        self.destiny: str = destiny

    def __str__(self) -> str:
        return f'{self.origin} -> {self.destiny}'        

class Container(containers.DeclarativeContainer):
    uber = providers.Factory(Uber)
    passenger = providers.Factory(Passenger)
    
@inject
def main(uber: Uber, 
         passenger: Passenger) -> None:
    print(f'Last go {passenger} the {uber}')

uber = Uber('sedan-luxo', 5)
passenger = Passenger('Campinas', 'Monte Mor')

container = Container()
container.wire(modules=[__name__])

main(uber, passenger)